На Kaggle мне удалось найти датасет, содержащий комментарии, размеченные на токсичные и обычные (1 и 0). 
Воспользуемся им для обучения модели для будущего бота.

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('labeled.csv')

data.tail()

In [ ]:
data.shape

Проверим наличие пропусков в данных:

In [ ]:
data.isnull().any().any()

Приведем целевую переменную к целому типу:

In [ ]:
data['toxic'] = data['toxic'].astype('int32')
data.head()

Посмотрим на распределение наших данных по классам (токсичные нетоксичные комментарии).

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
sns.set()

In [ ]:
fig = plt.figure(figsize=(15, 10))
sns.histplot(data['toxic'].astype('str'), color='r')
plt.xticks()
plt.show()

Можем видеть, что классы распределены неравномерно -- число токсичных комментариев примерно в 2 раза меньше, чем обычных. А если быть точнее:

In [ ]:
data['toxic'].value_counts()

Проверим качество разметки, выведя несколько токсичных и нетоксичных комментариев.

In [ ]:
[print(text) for text in data[data['toxic'] == 0]['comment'][15:17]]

In [ ]:
[print(text) for text in data[data['toxic'] == 1]['comment'][15:17]]

Больше смотреть не на что, так что разделим данные на `train` и `test` и займемся обработкой.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = data['comment']
y = data['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from string import punctuation, digits, ascii_lowercase

За стоп-символы будем считать знаки пунктуации, числа и английский алфавит (так как слова на английском практически не встречаются, и с их отсутствием смысл не меняется. При этом, немало сэмплов, где имеются ссылки и пр., но из-за пробелов их не так легко удалить регулярками). Также удалим слова, которые в русском языке используются очень часто при помощи `stopwords.words('russian')`

Напишем свой кастомный tokenizer, который будет:
1. Приводить текст к нижнему регистру
2. Разбивать текст на токены
3. Удалять токены, содержащие стоп-символы или являющиеся стоп-словами
4. Проводить стемминг при помощи SnowballStemmer

In [ ]:
stop_symb = punctuation + digits + ascii_lowercase
stemmer = SnowballStemmer("russian")
sw = stopwords.words('russian')

In [ ]:
def custom_tokenizer(s):
    s = [stemmer.stem(word) for word in word_tokenize(s.lower()) if word not in sw and word.translate(str.maketrans('', '', stop_symb)) == word]
    return s

Посмотрим на примере, что делает `custom_tokenizer`:

In [ ]:
num_sample = 7
print(f"Исходный сэмпл: {X_train.iloc[num_sample]}\nРезультат: {custom_tokenizer(X_train.iloc[num_sample])}")

Теперь токенизируем наши данные, используя `TfidfVectorizer` и наш катосмный токенайзер.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
%%time
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)
X_train = vectorizer.fit_transform(X_train)
X_train.shape

Взглянем на топ слов в словаре:

In [ ]:
import itertools

dict(itertools.islice(vectorizer.vocabulary_.items(), 10))

In [ ]:
%%time
X_test = vectorizer.transform(X_test)
X_test.shape

Настало время обучить модель. Для начала я выбрал логистическую регрессию, так как она неплохо справляется с классификацией и легко обучается.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf_lr = LogisticRegression(C=1.2)
clf_lr.fit(X_train, y_train)
y_predicted = clf_lr.predict(X_test)

Проверим работоспособность на отдельных примерах:

In [ ]:
def is_toxic(comment, clf=clf_lr, vectorizer=vectorizer):
    print("Нетоксичный") if clf.predict(vectorizer.transform([comment]))[0] == 0 else print("Токсичный")

In [ ]:
is_toxic('Как дела?')

In [ ]:
is_toxic('Ты дурак')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Посмотрим на метрики:

In [ ]:
def make_metrics(y_test, y_predicted):
    print(f"Метрики:\nAccuracy: {accuracy_score(y_test, y_predicted)}\nPrecision: {precision_score(y_test, y_predicted)}")
    print(f"Recall: {recall_score(y_test, y_predicted)}\nF1: {f1_score(y_test, y_predicted)}")

In [ ]:
make_metrics(y_test, y_predicted)

Как можно видеть, `recall` слишком низкий, в нашей задаче, пусть неточно, но лучше удалять больше комментариев, чем с большей точностью, но пропускать токсичные. Попробуем выбрать порог, где `precision` будет пониже, но еще вполне допустимый -- например, `0.8`:

In [ ]:
from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve

In [ ]:
plot_precision_recall_curve(estimator=clf_lr, X=X_test, y=y_test)

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_true=y_test, probas_pred=clf_lr.predict_proba(X_test)[:, 1])

In [ ]:
probas_pred = clf_lr.predict_proba(X_test)[:, 1] > thresholds[np.where(precision >= 0.8)[0][0]]

Посмотрим снова на метрики:

In [ ]:
make_metrics(y_test, probas_pred)

Видим, что `recall` заметно возрос (0.6485$ \rightarrow$ 0.8146), при это также выросли `accuracy` и `f1`.

Построим также матрицу ошибок:

In [ ]:
from sklearn.metrics import plot_confusion_matrix

In [ ]:
plot_confusion_matrix(clf_lr, X_test, y_test)

Видим, что у нас 330 **False Negative** значений, то есть тех комментариев, которые являются токсичными, но модель их определила, как обычные. Это, конечно, не самый лучший показатель.

Попробуем воспользоваться методом опорных векторов. Он обучается дольше, но справляться должен лучше.

In [ ]:
from sklearn.svm import SVC

In [ ]:
clf_svm = SVC()
clf_svm.fit(X_train, y_train)
y_predicted = clf_svm.predict(X_test)

In [ ]:
make_metrics(y_test, y_predicted)

**SVM** с дефолтными настройкми справился немного лучше, чем логистическая регрессия. Попробуем подобрать порог.

Обучим заново классификатор на SVM, но добавим параметр `probability=True` для подсчета вероятностей:

In [ ]:
clf_svm = SVC(C=1.2, probability=True)
clf_svm.fit(X_train, y_train)
y_predicted = clf_svm.predict(X_test)

In [ ]:
plot_precision_recall_curve(estimator=clf_svm, X=X_test, y=y_test)

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_true=y_test, probas_pred=clf_svm.predict_proba(X_test)[:, 1])

In [ ]:
probas_pred = clf_svm.predict_proba(X_test)[:, 1] > thresholds[np.where(precision >= 0.8)[0][0]]

In [ ]:
make_metrics(y_test, probas_pred)

In [ ]:
plot_confusion_matrix(clf_svm, X_test, y_test)

Метрики особо не изменились по сравнению с логистической регрессией, но зато **FN** значений стало меньше! Так же увеличилось число правильно распознанных токсичных комментариев.

Попробуем подобрать гиперпараметры получше для нашего классификатора.

Мне лень заново грид серч запускать, но там правда не очень (можно посмотреть предыдущие коммиты:))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf', 'poly', 'sigmoid']}

In [ ]:
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(X_train,y_train)

In [ ]:
print(grid.best_estimator_)

In [ ]:
clf_svm = SVC(C=10, gamma=0.1, probability=True)
clf_svm.fit(X_train, y_train)
y_predicted = clf_svm.predict(X_test)

In [ ]:
plot_precision_recall_curve(estimator=clf_svm, X=X_test, y=y_test)

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_true=y_test, probas_pred=clf_svm.predict_proba(X_test)[:, 1])
probas_pred = clf_svm.predict_proba(X_test)[:, 1] > thresholds[np.where(precision >= 0.8)[0][0]]

In [ ]:
make_metrics(y_test, probas_pred)

In [ ]:
plot_confusion_matrix(clf_svm, X_test, y_test)

До изменения порога модель и правда выдала более лучший результат, но с нужным нам значением `precision` остальные метрики заметно хуже, чем у классификатора до подбора гиперпараметров. Хотя стоит учесть, что число **FN** значений снова заметно уменьшилось и число правильно распознанных токсичных комментариев вновь увеличилось.

*Возможно, в силу небольшого размера датасета и неизвестного качества разметки добиться колоссального улучшения не получится, используя методы классического машинного обучения.*

**Но можно попробовать разбить длинные комментарии на два отдельных. При этом обоим частям будем присваивать метку исходного текста. А также будем надеяться, что это действительно так:) Ведь скорее всего в длинных комментариях окраска сохраняется на всем протяжении, а не в единственном предложении.**

Создадим маску для комментариев, содержащих как минимум 40 слов:

In [ ]:
data_mask = data['comment'].apply(lambda x: len(x.split(' ')) >= 40)

In [ ]:
long_data = data[data_mask]
long_data.shape

Оказывается, что такие комментарии составляют почти 20%!

In [ ]:
ind_split = long_data['comment'].apply(lambda x: len(' '.join(x.split(' ')[:len(x.split(' ')) // 2])))
split_data = long_data.copy()
new_comm = []
for index, row in long_data.iterrows():
    split_data.at[index, 'comment'] = row['comment'][:ind_split[index]]
    new_comm.append(row['comment'])
new_data = pd.DataFrame({'comment': new_comm, 'toxic': split_data['toxic']})

In [ ]:
data[data_mask] = split_data
data = pd.concat([data, new_data], ignore_index=True)

Посмотрим на новый размер датасета:

In [ ]:
data.shape

Теперь повторим все для новых данных...

In [ ]:
X = data['comment']
y = data['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
%%time
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)
X_train = vectorizer.fit_transform(X_train)
X_train.shape

In [ ]:
%%time
X_test = vectorizer.transform(X_test)
X_test.shape

In [ ]:
clf_svm = SVC(C=1.2, probability=True)
clf_svm.fit(X_train, y_train)
y_predicted = clf_svm.predict(X_test)

In [ ]:
make_metrics(y_test, y_predicted)

In [ ]:
plot_precision_recall_curve(estimator=clf_svm, X=X_test, y=y_test)

Судя по PR-кривой, мы можем поднять порог до `precision`$=0.85$:

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_true=y_test, probas_pred=clf_svm.predict_proba(X_test)[:, 1])
probas_pred = clf_svm.predict_proba(X_test)[:, 1] > thresholds[np.where(precision >= 0.85)[0][0]]

In [ ]:
make_metrics(y_test, probas_pred)

In [ ]:
plot_confusion_matrix(clf_svm, X_test, y_test)

Результат налицо -- при повышенном `precision` мы получили  `recall`$=0.84$! Мы стали распознавать правильно гораздо больше токсичных комментариев (и нетоксичных тоже)!

*Возможно, стоит попробовать еще посплитить комментарии, так как в датасете все еще остались слишком длинные.*

Теперь проведем обучение на всем датасете, чтобы сохранить обученную модель и векторайзер уже для использования в боте!

In [ ]:
%%time
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)
X = vectorizer.fit_transform(X)
X.shape

In [ ]:
clf_svm = SVC(C=1.2, probability=True)
clf_svm.fit(X, y)

In [ ]:
import pickle
pickle.dump(clf_svm, open('pretrained_clf', 'wb'))

In [ ]:
pickle.dump(vectorizer, open('pretrained_vect', 'wb'))

Проверим, что сохраненная модель корректно работает:

In [ ]:
loaded_model = pickle.load(open('pretrained_clf', 'rb'))
loaded_vect = pickle.load(open('pretrained_vect', 'rb'))

In [ ]:
is_toxic('Что делаешь?', clf=loaded_model, vectorizer=loaded_vect)